# **VQGAN_CLIP_GENERATOR**
Documentation is provided at [the project home page](https://github.com/rkhamilton/vqgan-clip-generator).


In [1]:
#@markdown What GPU am I using?

#@markdown V100 > P100 > everything else

!nvidia-smi --query-gpu=gpu_name,memory.used,memory.total,gpu_bus_id,vbios_version --format=csv
gpu_name = !nvidia-smi --query-gpu=gpu_name, --format=csv

name, memory.used [MiB], memory.total [MiB], pci.bus_id, vbios_version
Tesla P100-PCIE-16GB, 0 MiB, 16280 MiB, 00000000:00:04.0, 86.00.52.00.02


# **Filesystem Setup**
Run below to set your root location.

In [1]:
#@title Use Google Drive as your Filesystem
#@markdown This option connects to your google drive, and saves model files and outputs to that location for re-use.
import os
abs_root_path = "/content"

from google.colab import drive
drive.mount('/content/drive')

def ensureProperRootPath():
    if len(abs_root_path) > 0:
        os.chdir(abs_root_path) # Changes directory to absolute root path
        print("Root path check: ")
        !pwd

ensureProperRootPath()

def make_folder(folder_name):
  if len(folder_name) > 0:
      path_tmp = os.path.join("/content/drive/MyDrive/",folder_name)
      if not os.path.exists(path_tmp):
          os.mkdir(path_tmp)
      return path_tmp


project_folder_name = "video-style-transfer" #@param {type: "string"}
abs_root_path = make_folder(project_folder_name)

print("Created folder & set root path to: " + abs_root_path)
%cd $abs_root_path
!git clone https://github.com/dpanangian/video-style-transfer.git .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root path check: 
/content
Created folder & set root path to: /content/drive/MyDrive/video-style-transfer
/content/drive/MyDrive/video-style-transfer
fatal: destination path '.' already exists and is not an empty directory.


# **Download Dependencies for Google Colab**

In [3]:
#@title Download and install dependencies
#@markdown Needed every time you connect to a fresh colab runtime.

#@markdown Be aware that this can take five minutes or so as it downloads a few gigabytes of libraries to your Colab runtime environment.
%cd $abs_root_path/few-shot-patch-based-training
!python setup.py develop
%cd $abs_root_path
!python setup.py develop
!pip install git+https://github.com/openai/CLIP.git taming-transformers==0.0.1 ftfy==6.0.3 regex pytorch-lightning==1.4.9 kornia==0.5.11 imageio==2.9.0 omegaconf==2.1.1 torch-optimizer==0.1.0 piexif==1.1.3
!pip install lpips
!pip install opencv-python scipy
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install git+https://github.com/xinntao/Real-ESRGAN
!pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
!pip install -r requirements.txt

/content/drive/MyDrive/video-style-transfer/few-shot-patch-based-training
running develop
running egg_info
writing src/few_shot_patch_based_training.egg-info/PKG-INFO
writing dependency_links to src/few_shot_patch_based_training.egg-info/dependency_links.txt
writing requirements to src/few_shot_patch_based_training.egg-info/requires.txt
writing top-level names to src/few_shot_patch_based_training.egg-info/top_level.txt
writing manifest file 'src/few_shot_patch_based_training.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/few-shot-patch-based-training.egg-link (link to src)
Adding few-shot-patch-based-training 0.0.0 to easy-install.pth file

Installed /content/drive/MyDrive/video-style-transfer/few-shot-patch-based-training/src
Processing dependencies for few-shot-patch-based-training==0.0.0
Searching for torchvision==0.7.0
Reading https://pypi.org/simple/torchvision/
Best match: torchvision 0.7.0
Processing torchvision-0.7.0-cp37-cp37m-manylinux

  Cloning https://github.com/xinntao/Real-ESRGAN to /tmp/pip-req-build-c86so_df
  Running command git clone -q https://github.com/xinntao/Real-ESRGAN /tmp/pip-req-build-c86so_df
  Created wheel for realesrgan: filename=realesrgan-0.2.4.0-py3-none-any.whl size=25984 sha256=2d787c0366bf4ee624a8f253d250c030db42ed74ea239b933bf3b5e9d9bbcc13
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzqeuodt/wheels/d4/39/50/3b22ae424571c09f27b6a10f7241eb5f2cb7d5c2481b7aeded
Successfully built realesrgan
Looking in links: https://download.pytorch.org/whl/torch/, https://download.pytorch.org/whl/torchvision/
     |████████████████████████████████| 831.4 MB 2.9 kB/s 
     |████████████████████████████████| 7.6 MB 3.8 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing ins

# **Define Paths and Import libraries**
Run this once every time you reset your runtime.

In [2]:
#@title Paths
import os
extracted_video_frames_dir = os.path.join(abs_root_path, "extracted_video_frames")
generated_video_frames_path = os.path.join(abs_root_path, "generated_video_frames")
upscaled_video_frames_path = os.path.join(abs_root_path, "upscaled_video_frames")

output_dir = os.path.join(abs_root_path, "output")
models_dir = os.path.join(abs_root_path, "models")

make_folder(extracted_video_frames_dir)
make_folder(generated_video_frames_path)
make_folder(upscaled_video_frames_path)
make_folder(output_dir)
make_folder(models_dir)

print(f'abs_root_path:\t\t\t{abs_root_path}')
print(f'output_dir:\t\t\t{output_dir}')
print(f'generated_video_frames_path:\t{generated_video_frames_path}')
print(f'extracted_video_frames_dir:\t{extracted_video_frames_dir}')
print(f'upscaled_video_frames_path:\t{upscaled_video_frames_path}')
print(f'models_dir:\t\t\t{models_dir}')


abs_root_path:			/content/drive/MyDrive/video-style-transfer
output_dir:			/content/drive/MyDrive/video-style-transfer/output
generated_video_frames_path:	/content/drive/MyDrive/video-style-transfer/generated_video_frames
extracted_video_frames_dir:	/content/drive/MyDrive/video-style-transfer/extracted_video_frames
upscaled_video_frames_path:	/content/drive/MyDrive/video-style-transfer/upscaled_video_frames
models_dir:			/content/drive/MyDrive/video-style-transfer/models


In [6]:
#@title Libraries
from vqgan_clip import generate, esrgan, video_tools
from vqgan_clip.engine import VQGAN_CLIP_Config
import os
from vqgan_clip import _functional as VF

from IPython import display
import subprocess
from tqdm.auto import tqdm

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/vgg.pth


# **Generate a single image from a prompt**
This section lets you generate an image using any combination of text prompts (most common) and image prompts. An initial image may also be uploaded to your project root folder and used.

In [ ]:
#@title Generate the image
config = VQGAN_CLIP_Config()
config.model_dir = models_dir
output_image_width_pixels = 600 #@param {type: "integer"}
output_image_height_pixels =  800#@param {type: "integer"}
loss = ['mse-pixel','lpips']
weight = [0.8,0.9]
# Set True if you instal
config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
config.init_image_method = loss
text_prompts = 'Fernando Botero Painting' #@param {type: "string"}
init_image="images/init_image.jpg" #@param {type: "string"}
iterations =  120 #@param {type: "integer"}
upscale_image = False 
image_prompts = None

output_filename = os.path.join(output_dir,'my_image.jpg')

metadata_comment = generate.image(eng_config=config,
                                init_image=init_image,
                                init_weight=weight,
                                save_every=10,
                                text_prompts=text_prompts,
                                image_prompts=image_prompts,
                                iterations=iterations,
                                output_filename=output_filename,
                                  verbose=False)


display.display(display.Image(output_filename))



# **Style transfer to an existing video with patch-based training**
Apply a VQGAN+CLIP style to an existing video. Sequential / parallel prompts are supported (e.g. you can change from prompt 1 to prompt 2 at frame 30). You must balance the iterations_per_frame and current_source_frame_image_weight. See [the documentation](https://github.com/rkhamilton/vqgan-clip-generator#style-transfer) for tips.

In [17]:
#@title Setup paths and parameters
#@markdown An existing video will have a VQGAN style applied to each frame.
#@markdown Refer to the [documentation](https://github.com/rkhamilton/vqgan-clip-generator/blob/main/README.md) 
#@markdown for an explanation of the options.

config = VQGAN_CLIP_Config()
config.model_dir = models_dir
input_video_path = 'videos/potrait_Trim.mp4' #@param {type: "string"}
output_video_filename = 'style_transfer_video.mp4' #@param {type: "string"}
extraction_framerate = 30 #@param {type: "integer"}
video_exraction_framerate = extraction_framerate
text_prompts = 'Fernando Botero Painting' #@param {type: "string"}
output_image_width_pixels =  800#@param {type: "integer"}
output_image_height_pixels =  600#@param {type: "integer"}
iterations_per_frame =  120#@param {type: "integer"}

current_source_frame_image_weight =[0.8,0.9]

current_source_frame_prompt_weight=0.0 

config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
init_image = f'{output_dir}{os.sep}init_image.jpg'
output_video_path = f'{output_dir}{os.sep}{output_video_filename}'
video_frames_to_encode = generated_video_frames_path
metadata_comment = ''
generated_video_no_audio=f'{output_dir}{os.sep}output_no_audio.mp4'

upscale_images = False
face_enhance = False 
interpolate_with_RIFE = False


copy_audio = False 


interpolation_with_RIFE = False

config.seed = 111
config.init_image_method = ['mse','lpips']
image_prompts = None

output_root_dir = 'example media'
# Generated video framerate. Images will be extracted from the source video at this framerate, using interpolation if needed.
final_video_filename = os.path.join(output_dir, 'style transfer.mp4')

fewshot_frames=3 #@param {type: "integer"}
fewshot_iterations = 5000#@param {type: "integer"}


In [ ]:
#@title Style Tranfer



# Set some paths
generated_video_frames_path = os.path.join(output_root_dir, 'generated video frames')
upscaled_video_frames_path = os.path.join(output_root_dir, 'upscaled video frames')

# Use a wrapper for FFMPEG to extract stills from the original video.
original_video_frames = video_tools.extract_video_frames(input_video_path,
                                                         extraction_framerate=video_exraction_framerate,
                                                         extracted_video_frames_path=os.path.join(output_root_dir, 'extracted video frames'))

# Apply a style to the extracted video frames.
metadata_comment = generate.style_transfer_fewshot(original_video_frames,
                                           video_frames_path=os.path.join(output_root_dir, 'extracted video frames'),
                                           output_path= generated_video_frames_path,
                                           fewshot_config_file='few-shot-patch-based-training/_config/reference_P.yaml',
                                           eng_config=config,
                                           current_source_frame_image_weight=current_source_frame_image_weight,
                                           current_source_frame_prompt_weight=current_source_frame_prompt_weight,
                                           text_prompts=text_prompts,
                                           image_prompts=image_prompts,
                                           iterations_per_frame=iterations_per_frame,
                                           verbose=True,
                                           fewshot_frames=fewshot_frames,
                                           fewshot_iterations = fewshot_iterations
                                           )
if upscale_images:
    esrgan.inference_realesrgan(input=generated_video_frames_path,
                                output_images_path=upscaled_video_frames_path,
                                face_enhance=face_enhance,
                                purge_existing_files=True,
                                # model_filename='RealESRGAN_x4plus_anime_6B.pth',
                                # model_url='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth',
                                netscale=4,
                                outscale=4)
    # copy metadata from generated images to upscaled images.
    VF.copy_image_metadata(generated_video_frames_path, upscaled_video_frames_path)
    video_frames_to_encode = upscaled_video_frames_path
else:
    video_frames_to_encode = generated_video_frames_path


# Use a wrapper for FFMPEG to encode the video. Try setting video_encode_framerate=video_exraction_framerate/2 for a slow motion look.
generated_video_no_audio = f'example media{os.sep}output no audio.mp4'
video_tools.encode_video(output_file=generated_video_no_audio,
                         path_to_stills=video_frames_to_encode,
                         metadata_title=text_prompts,
                         metadata_comment=metadata_comment,
                         input_framerate=video_exraction_framerate, png=True)

if copy_audio:
    # Copy audio from the original file
    video_tools.copy_video_audio(
        input_video_path, generated_video_no_audio, final_video_filename)
    os.remove(generated_video_no_audio)
else:
    if os.path.exists(final_video_filename):
        os.remove(final_video_filename)
    os.rename(generated_video_no_audio, final_video_filename)

if interpolate_with_RIFE:
    video_tools.RIFE_interpolation(input=final_video_filename,
                       output=f'{os.path.splitext(final_video_filename)[0]}_RIFE.mp4',
                       interpolation_factor=4,
                       metadata_title=text_prompts,
                       metadata_comment=metadata_comment)




style transfer:   0%|          | 0/3 [00:00<?, ?image/s]

single image:   0%|          | 0/120 [00:00<?, ?iteration/s]

single image:   0%|          | 0/120 [00:00<?, ?iteration/s]

In [16]:
#@title Download video from Google Colab using the browser
#@markdown If you are using the Google Drive filesystem, the file is already 
#@markdown stored there. Running this block lets you download it through your browser.
from google.colab import files
files.download(final_video_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>